In [1]:
import json

In [4]:
 # Opening JSON file
with open('/home/minami/Code/Git-repo/Mask-RCNN/data/train/categories.json') as f:
    data = json.load(f)

In [6]:
data

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},

In [21]:
import cv2
import numpy as np

for index in (4,2,5,326,6,756,6796,4568,432,235):
    id = data['annotations'][index]['image_id']
    bbox = data['annotations'][index]['bbox']
    for image_info in data['images']:
        if image_info['id'] == id:
            img_name = image_info['file_name']
            height = image_info['height']
            width = image_info['width']
            mask = np.zeros((height,width))
    
    img = cv2.imread('data/coco/train2014/' +img_name)
    output = img.copy()
    print(data['annotations'][index]['iscrowd'], img.shape)
    cv2.rectangle(output,(int(bbox[0]),int(bbox[1])),(int(bbox[0]+bbox[2]),int(bbox[1]+bbox[3])),(0,255,0),3)
    ann = data['annotations'][index]
    rle = annToRLE(ann, height, width)
    mask = maskUtils.decode(rle)
    mask = np.stack((mask,)*3, axis=-1)
    mask = np.where(mask>0,(255,255,255),img)
    mask = mask.astype(np.uint8)
    cv2.addWeighted(mask, 0.5, output, 1 - 0.5,
		0, output)
    cv2.imshow('mask',mask)
    cv2.imshow('img',output)
    cv2.waitKey(0)
cv2.destroyAllWindows()

0 (468, 640, 3)
0 (385, 500, 3)
0 (500, 333, 3)
0 (423, 500, 3)
0 (426, 640, 3)
0 (480, 640, 3)
0 (427, 640, 3)
0 (427, 640, 3)
0 (394, 500, 3)
0 (640, 480, 3)


In [5]:
from pycocotools import mask as maskUtils

In [8]:
# The following two functions are from pycocotools with a few changes.

def annToRLE(ann, height, width):
    """
    Convert annotation which can be polygons, uncompressed RLE to RLE.
    :return: binary mask (numpy 2D array)
    """
    segm = ann['segmentation']
    if isinstance(segm, list):
        # polygon -- a single object might consist of multiple parts
        # we merge all parts into one mask rle code
        rles = maskUtils.frPyObjects(segm, height, width)
        rle = maskUtils.merge(rles)
    elif isinstance(segm['counts'], list):
        # uncompressed RLE
        rle = maskUtils.frPyObjects(segm, height, width)
    else:
        # rle
        rle = ann['segmentation']
    return rle

def annToMask(ann, height, width):
    """
    Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
    :return: binary mask (numpy 2D array)
    """
    rle = annToRLE(ann, height, width)
    m = maskUtils.decode(rle)
    return m

In [12]:
ann = data['annotations'][index]
rle = annToRLE(ann, height, width)
m = maskUtils.decode(rle)

In [20]:
a = np.where(m>0,255,0)
a = a.astype(np.uint8)
cv2.imshow('img',a)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
from pycocotools.coco import COCO

In [6]:
coco = COCO("/home/minami/Code/Git-repo/Mask-RCNN/data/coco/annotations_trainval2014/annotations/instances_train2014.json")

loading annotations into memory...
Done (t=17.58s)
creating index...
index created!


In [7]:
class_ids = sorted(coco.getCatIds())

In [9]:
# All images or a subset?
if class_ids:
    image_ids = []
    for id in class_ids:
        image_ids.extend(list(coco.getImgIds(catIds=[id])))
    # Remove duplicates
    image_ids = list(set(image_ids))
else:
    # All images
    image_ids = list(coco.imgs.keys())

In [13]:
image_ids = []
for id in class_ids:
    image_ids.extend(list(coco.getImgIds(catIds=[id])))
# Remove duplicates
image_ids = list(set(image_ids))
len(image_ids)

82081

In [14]:
# Add classes
class_info = [{"source": "", "id": 0, "name": "BG"}]
for i in class_ids:
    class_info.append({
            "source": "coco",
            "id": i,
            "name": coco.loadCats(i)[0]["name"],
        })

In [16]:
# Map sources to class_ids they support
sources = list(set([i['source'] for i in class_info]))
source_class_ids = {}
# Loop over datasets
for source in sources:
    source_class_ids[source] = []
    # Find classes that belong to this dataset
    for i, info in enumerate(class_info):
        # Include BG class in all datasets
        if i == 0 or source == info['source']:
            source_class_ids[source].append(i)

In [1]:
import skimage
from skimage import io
skimage.__version__

'0.16.2'